# Connect and Import

In [ ]:
# Connect google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import pickle
import time
import torch
import numpy as np
import torch.nn as nn
import multiprocessing
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from sklearn.metrics import roc_auc_score

# Dataset

Download dataset

In [ ]:
!cp /content/drive/MyDrive/deepfake_ds/ds_split_pkl.zip /content/
!unzip /content/ds_split_pkl.zip -d /content/ds_split

In [ ]:
# Dataset class definition

class VideoROIDataset(Dataset):
    def __init__(self, txt_file, pickle_dir, buffer_size=5, batch_size=100):
        self.pickle_dir = pickle_dir
        self.buffer_size = buffer_size                                          # N. of pkl files to load in the buffer
        self.batch_size = batch_size                                            # N. of video_data for each pkl batch
        self.lock = multiprocessing.Lock()                                      # Lock for sync

        # Read video name from .txt
        with open(txt_file, "r") as f:
            self.video_list = [line.strip().split(",")[0] for line in f]

        # Sort Pickle files
        self.pickle_files = sorted(
            [os.path.join(self.pickle_dir, f) for f in os.listdir(self.pickle_dir) if f.endswith(".pkl")],
            key=lambda x: int(x.split("_")[-1].split(".")[0])
        )
        self.total_videos = len(self.video_list)
        self.data = []
        self.current_file_idx = 0
        self._load_next_buffer()

    def _load_next_buffer(self):
        with self.lock:
            if len(self.data) > self.batch_size:                                # Buffer refresh check
                return

            if self.current_file_idx >= len(self.pickle_files):
                print("All pickle files loaded.")
                return

            # Load next N when buffer contains less than batch_size video_data
            files_loaded = 0
            while files_loaded < self.buffer_size - 1 and self.current_file_idx < len(self.pickle_files):
                file_path = self.pickle_files[self.current_file_idx]
                print(f"Loading file: {file_path}")
                with open(file_path, "rb") as f:
                    self.data.extend(pickle.load(f))
                self.current_file_idx += 1
                files_loaded += 1

            if not self.data:
                raise IndexError("No available data")

    def __getitem__(self, index):
        if not self.data or index >= len(self.data):
            self._load_next_buffer()                                            # Load new data if necessary
            if not self.data:
                raise IndexError("Buffer empty after loading!")

        video_data = self.data[index % len(self.data)]
        eyes = torch.stack(video_data["eyes"])
        nose = torch.stack(video_data["nose"])
        mouth = torch.stack(video_data["mouth"])
        label = torch.tensor(video_data["label"], dtype=torch.float32)

        return {"eyes": eyes, "nose": nose, "mouth": mouth, "label": label}

    def __len__(self):
        return self.total_videos

# Buffered dataloader class definition, for dinamic loading of batches
class BufferedDataLoader(DataLoader):
    def __init__(self, dataset, *args, **kwargs):
        super().__init__(dataset, *args, **kwargs)

    def __iter__(self):
        dataset = self.dataset                                                  # Dataset
        for batch in super().__iter__():
            # Check buffer dimension and load new data if necessary
            dataset._load_next_buffer()
            yield batch

In [ ]:
# Video list directories
train_txt = "/content/ds_split/train_ds.txt"
test_txt = "/content/ds_split/test_ds.txt"
val_txt = "/content/ds_split/val_ds.txt"

# Batch directories
train_batches_dir = "/content/ds_split/train_batches"
test_batches_dir = "/content/ds_split/test_batches"
val_batches_dir = "/content/ds_split/val_batches"

# Create Datasets
train_dataset = VideoROIDataset(train_txt, train_batches_dir)
test_dataset = VideoROIDataset(test_txt, test_batches_dir)
val_dataset = VideoROIDataset(val_txt, val_batches_dir)

# Create Dataloaders
dl_batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=dl_batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=dl_batch_size, shuffle=False, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=dl_batch_size, shuffle=False, num_workers=2, pin_memory=True)

print("DataLoaders created succesfully!")

# Model

## Model Definition

## Main

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class MultiInputMobileNetLSTM(nn.Module):
    def __init__(self, num_frames=20, num_classes=2, class_weights=None):
        super(MultiInputMobileNetLSTM, self).__init__()
        self.num_frames = num_frames

        # Backbone MobileNetV2 for each ROI
        self.backbone_eyes = self._initialize_mobilenet()
        self.backbone_nose = self._initialize_mobilenet()
        self.backbone_mouth = self._initialize_mobilenet()

        # GAP Layer
        self.gap = nn.AdaptiveAvgPool2d((1, 1))

        # LSTM for time analysis
        feature_dim = 1280;
        self.lstm = nn.LSTM(input_size=feature_dim * 3, hidden_size=512, num_layers=1, batch_first=True)

        # Fully connected layer for classification
        self.fc = nn.Linear(512, 1)

        # Add class weights if given
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32)
        else:
            self.class_weights = None

    # Initialize MobileNetV2 with first frozen layers
    def _initialize_mobilenet(self):
        mobilenet = models.mobilenet_v2(pretrained=True, width_mult=1)

        # Freeze first 14 layers
        for idx, layer in enumerate(mobilenet.features):
            if idx < 14:
                for param in layer.parameters():
                    param.requires_grad = False

        return nn.Sequential(*mobilenet.features)


    def forward(self, eyes, nose, mouth):
        batch_size = eyes.size(0) // self.num_frames

        # Extract feature through backbone MobileNet
        eyes_features = self.backbone_eyes(eyes)  # [B*num_frames, 1280, 1, 1]
        nose_features = self.backbone_nose(nose)
        mouth_features = self.backbone_mouth(mouth)

        # Apply Global Average Pooling
        eyes_features = self.gap(eyes_features).squeeze(-1).squeeze(-1)  # [B*num_frames, 1280]
        nose_features = self.gap(nose_features).squeeze(-1).squeeze(-1)
        mouth_features = self.gap(mouth_features).squeeze(-1).squeeze(-1)

        # Resize to proper dimensions
        eyes_features = eyes_features.view(batch_size, self.num_frames, -1)  # [B, num_frames, 1280]
        nose_features = nose_features.view(batch_size, self.num_frames, -1)
        mouth_features = mouth_features.view(batch_size, self.num_frames, -1)

        # Concatenation
        ROI_features = torch.cat((eyes_features, nose_features, mouth_features), dim=2)  # [B, num_frames, 3840]

        # LSTM for time analysis
        lstm_out, _ = self.lstm(ROI_features)  # [B, num_frames, 512]

        # Use only last frame output for classification
        lstm_final_output = lstm_out[:, -1, :]  # [B, 512]

        # Fully connected layer
        logits = self.fc(lstm_final_output)  # [B, 1]

        return logits.squeeze()  # [B]


## Model upgrade

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class MultiInputMobileNetLSTM(nn.Module):
    def __init__(self, config, num_frames=20, num_classes=2, class_weights=None):
        super(MultiInputMobileNetLSTM, self).__init__()
        self.config = config
        self.num_frames = num_frames

        # Backbone MobileNetV2 for each ROI
        self.backbone_eyes = self._initialize_mobilenet()
        self.backbone_nose = self._initialize_mobilenet()
        self.backbone_mouth = self._initialize_mobilenet()

        # GAP Layer
        self.gap = nn.AdaptiveAvgPool2d((1, 1))

        # Batch Normalization after MobileNet
        self.bn_eyes = nn.BatchNorm1d(1280)
        self.bn_nose = nn.BatchNorm1d(1280)
        self.bn_mouth = nn.BatchNorm1d(1280)

        # Dropout after concatenation
        self.dropout_ROI = nn.Dropout(p=self.config["dropout1"])

        # Fully connected layer to reduce dimension after concatenation
        feature_dim = 1280
        self.fc_ROI = nn.Linear(feature_dim * 3, feature_dim)
        self.ln_pre_LSTM=nn.LayerNorm(feature_dim)

        # LSTM for time analysis
        self.lstm = nn.LSTM(input_size=feature_dim, hidden_size=512, num_layers=1, batch_first=True)

        # Layer Normalization pre LSTM
        self.ln_post_lstm = nn.LayerNorm(512)
        self.dropout_lstm = nn.Dropout(p=self.config["dropout2"])

        # Fully connected layer for classification
        self.fc = nn.Linear(512, 1)

        # Add class weights if given
        if class_weights is not None:
            self.class_weights = torch.tensor(class_weights, dtype=torch.float32)
        else:
            self.class_weights = None

    # Initialize MobileNetV2 for training
    def _initialize_mobilenet(self):
        if self.config["backbone_dim"] == 1:
            mobilenet = models.mobilenet_v2(pretrained=True, width_mult=1)

            # Freeze first 14 layers
            for idx, layer in enumerate(mobilenet.features):
                if idx < 14:
                    for param in layer.parameters():
                        param.requires_grad = False

            return nn.Sequential(*mobilenet.features)

        else:
            width_mult = self.config["backbone_dim"]
            mobilenet = models.mobilenet_v2(pretrained=False, width_mult=width_mult)

            # Full training (no pretrained)
            for param in mobilenet.parameters():
                param.requires_grad = True

            return nn.Sequential(*mobilenet.features)

    def forward(self, eyes, nose, mouth):
        batch_size = eyes.size(0) // self.num_frames

        # Extract feature through backbone MobileNet
        eyes_features = self.backbone_eyes(eyes)                              # [B*num_frames, 1280, 2, 2]
        nose_features = self.backbone_nose(nose)
        mouth_features = self.backbone_mouth(mouth)

        # Apply Global Average Pooling
        eyes_features = self.gap(eyes_features).squeeze(-1).squeeze(-1)       # [B*num_frames, 1280]
        nose_features = self.gap(nose_features).squeeze(-1).squeeze(-1)
        mouth_features = self.gap(mouth_features).squeeze(-1).squeeze(-1)

        # Apply BatchNorm
        if self.config["batch_norm"]:
            eyes_features = self.bn_eyes(eyes_features)                           # [B*num_frames, 1280]
            nose_features = self.bn_nose(nose_features)
            mouth_features = self.bn_mouth(mouth_features)

        # Resize to proper dimensions
        eyes_features = eyes_features.view(batch_size, self.num_frames, -1)   # [B, num_frames, 1280]
        nose_features = nose_features.view(batch_size, self.num_frames, -1)
        mouth_features = mouth_features.view(batch_size, self.num_frames, -1)

        # Concatenation and Dropout
        ROI_features = torch.cat((eyes_features, nose_features, mouth_features), dim=2)  # [B, num_frames, 1280 * 3]
        ROI_features = self.dropout_ROI(ROI_features)

        # Fully connected layer
        ROI_features = self.fc_ROI(ROI_features)                                # [B, num_frames, 1280]
        if self.config["layer_norm1"]:
            ROI_features = self.ln_pre_LSTM(ROI_features)

        # LSTM for time analysis
        lstm_out, _ = self.lstm(ROI_features)                            # [B, num_frames, 512]

        # Use only last frame output for classification
        lstm_final_output = lstm_out[:, -1, :]                                # [B, 512]
        if self.config["layer_norm2"]:
            lstm_final_output = self.ln_post_lstm(lstm_final_output)
        lstm_final_output = self.dropout_lstm(lstm_final_output)

        # Fully connected layer
        logits = self.fc(lstm_final_output)                                   # [B, 1]

        return logits.squeeze()                                        # [B]


# Train

## Utility

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0, reduction='mean'):              # if main class is 1 (fake), alfa> 0.5
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        # Calculate probabilities
        probas = torch.sigmoid(inputs)
        pt = probas * targets + (1 - probas) * (1 - targets)

        # Calculate loss
        loss = -self.alpha * (1 - pt) ** self.gamma * torch.log(pt + 1e-12)  # Evita log(0)

        # Reduction
        if self.reduction == 'mean':
            return loss.mean()
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss

# Early Stopping
class EarlyStopping:
    def __init__(self, patience=5):
        self.patience = patience
        self.counter = 0
        self.best_loss = float("inf")
        self.early_stop = False

    def __call__(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

## Train loop

### Main

In [ ]:
# Path for checkpoints
CHECKPOINT_PATH = "/content/drive/MyDrive/deepfake_ds/modeep_best_models/"
os.makedirs(CHECKPOINT_PATH, exist_ok=True)
train_session = 40                                                             # Manually change this parameter for each training session

loss_choice = "Focal"                                                            # Focal or BCE

# Parameters
ok_config = {
    "dropout1": 0.6,
    "dropout2": 0.8,
    "batch_norm": False,
    "layer_norm1": False,
    "layer_norm2": True,
    "backbone_dim": 1
}

# Model initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiInputMobileNetLSTM(ok_config)
model = model.to(device)

# Loss and Optimizer
if loss_choice == "Focal":
    criterion = FocalLoss(alpha=0.85, gamma=3, reduction='mean')
else:
    total_real = 826
    total_fake = 5414
    pos_weight = torch.tensor([total_real / total_fake], dtype=torch.float32).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

optimizer = Adam(model.parameters(), lr=5e-6, weight_decay=1e-2)             # Optimizer with L2 reg (weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=10,                                                               # Numero di epoche per completare il ciclo
    eta_min=1e-7                                                            # Valore minimo del learning rate
)


# Training configuration
num_epochs = 30
early_stopping = EarlyStopping(patience=7)                                    # Early stopping

# Metrics lists
train_losses = []
val_losses = []
val_accuracies = []
val_aucs = []

# Training loop
best_val_loss = float("inf")
best_val_auc = 0.0

for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0

    for batch_idx, batch in enumerate(train_loader):

        # Load data
        eyes = batch["eyes"].view(-1, 3, 64, 64).to(device)
        nose = batch["nose"].view(-1, 3, 64, 64).to(device)
        mouth = batch["mouth"].view(-1, 3, 64, 64).to(device)
        labels = batch["label"].float().to(device)

        # Forward pass
        outputs = model(eyes, nose, mouth)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] Training Loss: {avg_train_loss:.5f}")

    # Validation
    model.eval()
    total_correct = 0
    total_samples = 0
    val_loss = 0.0
    all_labels = []                                                             # AUC
    all_probs = []                                                              # AUC

    with torch.no_grad():
        for val_batch in val_loader:

            # Load data
            eyes = val_batch["eyes"].view(-1, 3, 64, 64).to(device)
            nose = val_batch["nose"].view(-1, 3, 64, 64).to(device)
            mouth = val_batch["mouth"].view(-1, 3, 64, 64).to(device)
            labels = val_batch["label"].float().to(device)

            # Forward pass
            outputs = model(eyes, nose, mouth)
            val_loss += criterion(outputs, labels).item()

            # Save probabilities and labels for AUC
            probs = torch.sigmoid(outputs)
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

            # Accuracy
            predicted = (probs > 0.5).long()
            total_correct += (predicted == labels.long()).sum().item()
            total_samples += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = total_correct / total_samples
    val_auc = roc_auc_score(all_labels, all_probs)                 # AUC calculation

    print(f"Epoch [{epoch+1}/{num_epochs}] Validation Loss: {avg_val_loss:.5f}, "
          f"Accuracy: {val_accuracy:.4f}, AUC: {val_auc:.4f}")

    # Scheduler step
    scheduler.step()

    # Print learning rate
    for param_group in optimizer.param_groups:
        print(f"Learning Rate: {param_group['lr']}")

    # Save metrics
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    val_accuracies.append(val_accuracy)
    val_aucs.append(val_auc)

    # Save model if it's better
    if avg_val_loss < best_val_loss:                                            # or val_auc > best_val_auc
        best_val_loss = avg_val_loss
        best_val_auc = val_auc
        torch.save(model.state_dict(), f"{CHECKPOINT_PATH}best_model_{train_session}.pth")
        print(f"Saved checkpoint: Validation Loss {best_val_loss:.5f}, AUC {best_val_auc:.4f}")

    # Early stopping
    early_stopping(avg_val_loss)
    if early_stopping.early_stop:
        print("Activated early stopping!")
        break

    # Epoch time
    epoch_time = time.time() - start_time
    print(f"Time for epoch: {epoch_time / 60:.2f} minutes")



### Param tuning

In [ ]:
import itertools

# Parameters
param_grid = {
    "dropout1": [0.2, 0.5],
    "dropout2": [0.5, 0.7],
    "batch_norm": [False],
    "layer_norm1": [False],
    "layer_norm2": [True],
    "backbone_dim": [1]
}

# Create param combinations
keys, values = zip(*param_grid.items())
config_combinations = [dict(zip(keys, v)) for v in itertools.product(*values)]

In [ ]:
# Path for checkpoints
CHECKPOINT_PATH = "/content/drive/MyDrive/deepfake_ds/modeep_param_search/"
os.makedirs(CHECKPOINT_PATH, exist_ok=True)
train_session = 21                                                             # Manually change this parameter for each training session

loss_choice = "Focal"                                                            # Focal or BCE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for config in config_combinations:
    model = MultiInputMobileNetLSTM(config).to(device)

    # Loss and Optimizer
    if loss_choice == "Focal":
        criterion = FocalLoss(alpha=0.8, gamma=2.0, reduction='mean')
    else:
        total_real = 826
        total_fake = 5414
        pos_weight = torch.tensor([total_real / total_fake], dtype=torch.float32).to(device)
        criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    optimizer = Adam(model.parameters(), lr=1e-5, weight_decay=1e-2)             # Optimizer with L2 reg (weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.4, patience=1,
    )


    # Training configuration
    num_epochs = 40
    early_stopping = EarlyStopping(patience=5)                                    # Early stopping

    # Metrics lists
    train_losses = []
    val_losses = []
    val_accuracies = []
    val_aucs = []

    # Training loop
    best_val_loss = float("inf")
    best_val_auc = 0.0

    print(f"Training Session: {train_session}, Configuration:{config}")
    train_session += 1

    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()
        running_loss = 0.0

        for batch_idx, batch in enumerate(train_loader):

            # Load data
            eyes = batch["eyes"].view(-1, 3, 64, 64).to(device)
            nose = batch["nose"].view(-1, 3, 64, 64).to(device)
            mouth = batch["mouth"].view(-1, 3, 64, 64).to(device)
            labels = batch["label"].float().to(device)

            # Forward pass
            outputs = model(eyes, nose, mouth)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}] Training Loss: {avg_train_loss:.4f}")

        # Validation
        model.eval()
        total_correct = 0
        total_samples = 0
        val_loss = 0.0
        all_labels = []                                                             # AUC
        all_probs = []                                                              # AUC

        with torch.no_grad():
            for val_batch in val_loader:

                # Load data
                eyes = val_batch["eyes"].view(-1, 3, 64, 64).to(device)
                nose = val_batch["nose"].view(-1, 3, 64, 64).to(device)
                mouth = val_batch["mouth"].view(-1, 3, 64, 64).to(device)
                labels = val_batch["label"].float().to(device)

                # Forward pass
                outputs = model(eyes, nose, mouth)
                val_loss += criterion(outputs, labels).item()

                # Save probabilities and labels for AUC
                probs = torch.sigmoid(outputs)
                all_labels.extend(labels.cpu().numpy())
                all_probs.extend(probs.cpu().numpy())

                # Accuracy
                predicted = (probs > 0.5).long()
                total_correct += (predicted == labels.long()).sum().item()
                total_samples += labels.size(0)

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = total_correct / total_samples
        val_auc = roc_auc_score(all_labels, all_probs)                              # AUC calculation

        print(f"Epoch [{epoch+1}/{num_epochs}] Validation Loss: {avg_val_loss:.4f}, "
              f"Accuracy: {val_accuracy:.4f}, AUC: {val_auc:.4f}")

        # Scheduler step
        scheduler.step(avg_val_loss)

        # Print learning rate
        for param_group in optimizer.param_groups:
            print(f"Learning Rate: {param_group['lr']}")

        # Save metrics
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        val_accuracies.append(val_accuracy)
        val_aucs.append(val_auc)

        # Save model if it's better
        if avg_val_loss < best_val_loss:                                            # or val_auc > best_val_auc
            best_val_loss = avg_val_loss
            best_val_auc = val_auc
            torch.save(model.state_dict(), f"{CHECKPOINT_PATH}best_model_{train_session}.pth")
            print(f"Saved checkpoint: Validation Loss {best_val_loss:.4f}, AUC {best_val_auc:.4f}")

        # Early stopping
        early_stopping(avg_val_loss)
        if early_stopping.early_stop:
            print("Activated early stopping!")
            break

        # Epoch time
        epoch_time = time.time() - start_time
        print(f"Time for epoch: {epoch_time / 60:.2f} minutes")



## Test step

In [ ]:
CHECKPOINT_PATH = "/content/drive/MyDrive/deepfake_ds/modeep_best_models/"
train_session = 2

# Load weights
model.load_state_dict(torch.load(f"{CHECKPOINT_PATH}best_model_{train_session}.pth"))   # weights_only=True eventually
model.eval()
test_loss = 0.0
total_correct = 0
total_samples = 0

with torch.no_grad():
    for test_batch in test_loader:
        eyes = test_batch["eyes"].view(-1, 3, 64, 64).to(device)              # [batch_size, 20, 3, 64, 64] -> [batch_size*20, 3, 64, 64]
        nose = test_batch["nose"].view(-1, 3, 64, 64).to(device)
        mouth = test_batch["mouth"].view(-1, 3, 64, 64).to(device)
        labels = test_batch["label"].to(device)

        outputs = model(eyes, nose, mouth)
        test_loss += criterion(outputs, labels).item()

        predicted = (outputs > 0.5).long()
        total_correct += (predicted == labels.long()).sum().item()
        total_samples += labels.size(0)

avg_test_loss = test_loss / len(test_loader)
test_accuracy = total_correct / total_samples
print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Stats and Graphs

## Metrics plot

In [ ]:
import matplotlib.pyplot as plt

# Loss Graph
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

# Accuracy Graph
plt.figure(figsize=(10, 5))
plt.plot(val_accuracies, label="Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Validation Accuracy")
plt.legend()
plt.show()

# AUC Graph
plt.figure(figsize=(10, 5))
plt.plot(val_aucs, label="Validation AUC")
plt.xlabel("Epochs")
plt.ylabel("AUC")
plt.title("Validation AUC")
plt.legend()
plt.show()

## Model check

In [ ]:
import torch

CHECKPOINT_PATH = "/content/drive/MyDrive/deepfake_ds/modeep_best_models/"
train_session = 2
model_path = f"{CHECKPOINT_PATH}best_model_{train_session}.pth"

# Model load state
model = MultiInputMobileNetLSTM()
model.load_state_dict(torch.load(model_path))

# Conta i parametri
total_params = sum(p.numel() for p in model.parameters())
print(f"Il modello ha {total_params} parametri.")